In [1]:
my_collection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")
# 파이썬 리스트를 스파크 클러스터로 가져옴(파이썬 리스트 -> 스파크 RDD)
words = spark.sparkContext.parallelize(my_collection, 2)

In [2]:
supplementalData = {"Spark": 1000, "Definitive": 200, "Big": -300, "Simple": 100}

In [3]:
suppBroadcast = spark.sparkContext.broadcast(supplementalData)

In [4]:
# 브로드캐스트된 값 참조
suppBroadcast.value

{'Spark': 1000, 'Definitive': 200, 'Big': -300, 'Simple': 100}

In [6]:
words.map(lambda word: (word, suppBroadcast.value.get(word, 0)))\
.sortBy(lambda wordPair: wordPair[1])\
.collect()   

[('Big', -300),
 ('The', 0),
 ('Guide', 0),
 (':', 0),
 ('Data', 0),
 ('Processing', 0),
 ('Made', 0),
 ('Simple', 100),
 ('Definitive', 200),
 ('Spark', 1000)]

In [7]:
flights = spark.read\
.parquet("./data/flight-data/parquet/2010-summary.parquet")

In [8]:
# 프로그래밍 방식으로 집계 구하기

accChina = spark.sparkContext.accumulator(0)

In [9]:
def accChinaFunc(flight_row):
    destination = flight_row["DEST_COUNTRY_NAME"]
    origin = flight_row["ORIGIN_COUNTRY_NAME"]
    
    if destination == "China":
        accChina.add(flight_row["count"])
    if origin == "China":
        accChina.add(flight_row["count"])

In [10]:
#  foreach 메서드가 액션, 액션에서만 어큐뮬레이터의 실행을 보장
flights.foreach(lambda flight_row: accChinaFunc(flight_row))

In [11]:
# accumulator 값 조회
accChina.value

953